<a href="https://colab.research.google.com/github/zjzsu2000/CMPE297_Sec49AdvanceDL/blob/master/Assignment%202/CMPE297_Sec49_Assignment_2_b).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPE297_Sec49_Assignment_2_b)

Low level tensorflow code

In [1]:
import tensorflow as tf

## LinearClassifier

In [4]:
class LinearClassifier:
    def __init__(self, num_features=784, num_classes=2, learning_rate=0.01):
        self.num_classes = num_classes
        self.W = tf.Variable(tf.ones([num_features, num_classes]), name='weight')
        self.b = tf.Variable(tf.zeros([num_classes]), name='bias')
        self.optimizer = tf.optimizers.SGD(learning_rate)

    def calculate_y(self, x):
        y = tf.nn.softmax(tf.matmul(x, self.W) + self.b)
        return y

    def cost(self, y_pred, y_true):
        y_true = tf.one_hot(y_true, depth=self.num_classes)
        y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
        return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

    def iteration(self, x, y):
        with tf.GradientTape() as g:
            pred = self.calculate_y(x)
            loss = self.cost(pred, y)
        gradients = g.gradient(loss, [self.W, self.b])

        self.optimizer.apply_gradients(zip(gradients, [self.W, self.b]))


## SVMClassifier

In [5]:
class SVMClassifier:
    def __init__(self, num_features=784, num_classes=2, reg_strength=10000, learning_rate=0.01):
        self.num_classes = num_classes
        self.num_features = num_features
        self.W = tf.Variable(tf.ones([num_features, num_features]), name='weight')
        self.b = tf.Variable(tf.zeros([num_classes]), name='bias')
        self.reg_strength = reg_strength
        self.optimizer = tf.optimizers.SGD(learning_rate)

    def cost(self, x, y):
        distances = 1 - y * (tf.matmul(x, self.W))
        distances[distances < 0] = 0
        hinge_loss = self.reg_strength * (tf.reduce_sum(distances) / self.num_features)
        cost = 1 / 2 * tf.matmul(self.W, self.W) + hinge_loss
        return cost

    def iteration(self, x, y):
        with tf.GradientTape() as g:
            cost = self.cost(x, y)

        gradients = g.gradient(cost, self.W)

        self.optimizer.apply_gradients(zip(gradients, self.W))

## Neural Network

In [14]:
import keras


class DenseLayer(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)         

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name="kernel", shape=[batch_input_shape[-1], self.units], initializer="glorot_normal")
        self.bias = self.add_weight(name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units":self.units, "activation":keras.activations.serialize(self.activation)}


class NeuralNetwork:
    def __init__(self, layer1, layer2, layer3, layer4):
        self.layer1 = layer1
        self.layer2 = layer2        
        self.weight1 = self.layer1.kernel
        self.weight2 = self.layer2.kernel

        self.layer3 = layer3
        self.layer4 = layer4
        self.weight3 = self.layer3.kernel
        self.weight4 = self.layer4.kernel

    def back_propagation(self, x, y, iterations):
        for iteration in range(iterations):
            output1 = tf.sigmoid(tf.matmul(x, self.weight1))
            output2 = tf.sigmoid(tf.matmul(output1, self.weight2))
            output3 = tf.sigmoid(tf.matmul(output2, self.weight3))
            output4 = tf.sigmoid(tf.matmul(output3, self.weight4))

            layer4_error = y - output4
            layer4_delta = layer4_error * derivative(output4)
            layer3_error = tf.matmul(layer4_delta, tf.transpose(self.weight4))
            layer3_delta = layer3_error * derivative(output3)
            layer2_error = tf.matmul(layer3_delta, tf.transpose(self.weight3))
            layer2_delta = layer2_error * derivative(output2)
            layer1_error = tf.matmul(layer2_delta, tf.transpose(self.weight2))
            layer1_delta = layer1_error * derivative(output1)

            layer1_offset = tf.matmul(tf.transpose(x), layer1_delta)
            layer2_offset = tf.matmul(tf.transpose(output1), layer2_delta)
            layer3_offset = tf.matmul(tf.transpose(output2), layer3_delta)
            layer4_offset = tf.matmul(tf.transpose(output3), layer4_delta)

            self.weight1 += layer1_offset
            self.weight2 += layer2_offset
            self.weight3 += layer3_offset
            self.weight4 += layer4_offset

    # def backprop(self, x, y, iterations):
    #     for iteration in range(iterations):
    #         output1 = tf.sigmoid(tf.matmul(x, self.layer1.weights))
    #         output2 = tf.sigmoid(tf.matmul(output1, self.layer2.weights))
    #
    #         layer2_error = y - output2
    #         layer2_delta = layer2_error * derivative(output2)
    #
    #         layer1_error = tf.matmul(layer2_delta, tf.transpose(self.weight2))
    #         layer1_delta = layer1_error * derivative(output1)
    #
    #         layer1_offset = tf.matmul(tf.transpose(x), layer1_delta)
    #         layer2_offset = tf.matmul(tf.transpose(output1), layer2_delta)
    #
    #         self.weight1 += layer1_offset
    #         self.weight2 += layer2_offset


def derivative(x):
    return x * (1 - x)

if __name__ == '__main__':
    layer1 = DenseLayer(128)
    layer1.build((28,28,3))
    layer2 = DenseLayer(64)
    layer2.build((28,28,3))
    layer3 = DenseLayer(64)
    layer3.build((28,28,3))
    layer4 = DenseLayer(64)
    layer4.build((28,28,3))
    neura_network = NeuralNetwork(layer1, layer2, layer3, layer4)